In [50]:
import pandas as pd

def remove_accents(text):
    # Mapping of accented characters to their non-accented counterparts
    replacements = {
        'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U',
        'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
        'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U',
        'Ñ': 'N', 'ñ': 'n'
    }
    for accented_char, replacement in replacements.items():
        text = text.replace(accented_char, replacement)
    return text

In [51]:
data = pd.read_csv("ML-water/all_data_horte_Raque.csv")
data = data.drop(labels = ["Unnamed: 0", "index","std_impact"], axis = 1)
for col in ['NOM_ENT', 'NOM_MUN']:
    data[col] = data[col].apply(remove_accents)
    data[col] = data[col].astype(str).str.upper()
    data[col] = data[col].astype(str).str.strip()
quality_lenticos = pd.read_csv("wq_promedio_Lenticos.csv")
quality_costeros = pd.read_csv("wq_promedio_Costeros.csv")
quality_loticos = pd.read_csv("wq_promedio_Loticos.csv")
quality_subterraneos= pd.read_csv("wq_promedio_Subterraneo.csv")

In [53]:
# Step 1: Prepare all semaforo data from different sources
dfs = [
    quality_lenticos[["CVEGEO", "semaforo"]],
    quality_loticos[["CVEGEO", "semaforo"]],
    quality_subterraneos[["CVEGEO", "semaforo"]],
    quality_costeros[["CVEGEO", "semaforo"]]
]

# Step 2: Concatenate all semaforo info into one DataFrame
all_semaforos = pd.concat(dfs, ignore_index=True)

# Step 3: Group by CVEGEO and get the mode of semaforo
def mode_func(series):
    modes = series.mode()
    return modes.iloc[0] if not modes.empty else pd.NA

semaforo_mode = all_semaforos.groupby("CVEGEO")["semaforo"].apply(mode_func).reset_index()
semaforo_mode.head()

data = pd.merge(data, semaforo_mode, how = "left", on = "CVEGEO")
data.head()

,CVEGEO,NOM_ENT,NOM_MUN,POB_TOTAL,ALTITUD,AREA,UMBRAL12H,PORCENTA_1,urban,agua_AR,agua_FC,agua_PA,agua_PT,agua_PP,avg_impact,avg_agricultura,avg_construccion,avg_manufactura,avg_mineria,semaforo
0,1001,AGUASCALIENTES,AGUASCALIENTES,948990.0,1918.046196,116635.924756,57.80700,4.829056,3.0,91.0,226.0,2.0,24.0,0.0,0.900702,21.0,24.193751,44.979780,2.0,Rojo
1,1002,AGUASCALIENTES,ASIENTOS,51536.0,2030.520325,54337.426602,60.05970,4.274541,0.0,0.0,54.0,1.0,9.0,0.0,0.904023,0.0,4.500000,2.083333,0.0,Verde
2,1003,AGUASCALIENTES,CALVILLO,58250.0,1827.143813,92335.783227,60.71769,1.158972,0.0,7.0,43.0,1.0,4.0,0.0,0.787156,3.0,2.333333,2.835498,1.0,Rojo
3,1004,AGUASCALIENTES,COSIO,17000.0,1967.656566,12838.354453,52.22142,3.153082,0.0,10.0,13.0,1.0,3.0,0.0,0.912407,0.0,0.000000,1.000000,0.0,Rojo
4,1005,AGUASCALIENTES,JESUS MARIA,129929.0,1936.513072,49989.509156,58.46688,3.357602,3.0,34.0,42.0,1.0,16.0,0.0,0.798126,3.0,23.113636,80.622222,106.0,Rojo


In [55]:
print(data.isna().sum())

CVEGEO                 0
NOM_ENT                0
NOM_MUN                0
POB_TOTAL              6
ALTITUD                0
AREA                   0
UMBRAL12H             18
PORCENTA_1            18
urban                  0
agua_AR                0
agua_FC                0
agua_PA                0
agua_PT                0
agua_PP                0
avg_impact             0
avg_agricultura      117
avg_construccion     117
avg_manufactura      117
avg_mineria          117
semaforo            1429
dtype: int64


In [58]:
#data = data.dropna(subset = ["UMBRAL12H", "PORCENTA_1"])
data[["UMBRAL12H", "PORCENTA_1", "avg_agricultura", "avg_construccion", "avg_manufactura", "avg_mineria"]] = data[["UMBRAL12H", "PORCENTA_1", "avg_agricultura", "avg_construccion", "avg_manufactura", "avg_mineria"]].fillna(method="ffill")
data = data.dropna(subset=["POB_TOTAL"])
print(data.isna().sum())

CVEGEO                 0
NOM_ENT                0
NOM_MUN                0
POB_TOTAL              0
ALTITUD                0
AREA                   0
UMBRAL12H              0
PORCENTA_1             0
urban                  0
agua_AR                0
agua_FC                0
agua_PA                0
agua_PT                0
agua_PP                0
avg_impact             0
avg_agricultura        0
avg_construccion       0
avg_manufactura        0
avg_mineria            0
semaforo            1423
dtype: int64


C:\Users\amvc6\AppData\Local\Temp\ipykernel_16984\1814019549.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data[["UMBRAL12H", "PORCENTA_1", "avg_agricultura", "avg_construccion", "avg_manufactura", "avg_mineria"]] = data[["UMBRAL12H", "PORCENTA_1", "avg_agricultura", "avg_construccion", "avg_manufactura", "avg_mineria"]].fillna(method="ffill")


In [59]:
data.to_csv("AllData.csv", index=False)

In [49]:
data.head()

,CVEGEO,NOM_ENT,NOM_MUN,POB_TOTAL,ALTITUD,AREA,UMBRAL12H,PORCENTA_1,urban,agua_AR,agua_FC,agua_PA,agua_PT,agua_PP,avg_impact,avg_agricultura,avg_construccion,avg_manufactura,avg_mineria,semaforo
0,1001,AGUASCALIENTES,AGUASCALIENTES,948990.0,1918.046196,116635.924756,57.80700,4.829056,3.0,91.0,226.0,2.0,24.0,0.0,0.900702,21.0,24.193751,44.979780,2.0,Rojo
1,1002,AGUASCALIENTES,ASIENTOS,51536.0,2030.520325,54337.426602,60.05970,4.274541,0.0,0.0,54.0,1.0,9.0,0.0,0.904023,0.0,4.500000,2.083333,0.0,Verde
2,1003,AGUASCALIENTES,CALVILLO,58250.0,1827.143813,92335.783227,60.71769,1.158972,0.0,7.0,43.0,1.0,4.0,0.0,0.787156,3.0,2.333333,2.835498,1.0,Rojo
3,1004,AGUASCALIENTES,COSIO,17000.0,1967.656566,12838.354453,52.22142,3.153082,0.0,10.0,13.0,1.0,3.0,0.0,0.912407,0.0,0.000000,1.000000,0.0,Rojo
4,1005,AGUASCALIENTES,JESUS MARIA,129929.0,1936.513072,49989.509156,58.46688,3.357602,3.0,34.0,42.0,1.0,16.0,0.0,0.798126,3.0,23.113636,80.622222,106.0,Rojo
